In [ ]:
import torch
from torch.optim import Adam

In [ ]:
# dataset loading

In [ ]:
lr = 0.001
batch_size = 128  # not from the paper
binary_classification_criterion = torch.nn.BCELoss
regression_criterion = ... # "logarithm of the hyperbolic cosine" from the paper (?)

In [ ]:
def train(
        base_model,
        surrogate_model,
        dataset,
):
    optimizer = Adam(..., lr=lr)
    pass